In [112]:
import time
import calendar
import codecs
import datetime
import json
import sys
import gzip
import string
import glob
import re
import os
import numpy as np
from os import listdir
from os.path import isfile, join
import csv
import networkx as nx
from demjson import decode
import pandas as pd
import random
from collections import defaultdict
from nltk.corpus import stopwords
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score
from itertools import chain
import math

In [173]:
globalTweetCounter = 0

timeFormat = "%Y-%m-%dT%H:%M:%S.%fZ"

In [174]:
countries=['SAfrica-community-relevant-restricted.json']

for m in range(len(countries)):
    file_string=countries[m]
    mypath='/Users/tomfw/Downloads/DataShared/'+file_string
    G=nx.Graph()
    tweetList = []
    timeList = []
    userList = []
    
    with open(mypath) as f1:
        for line in f1:
           if random.random() < 2: # < 1 if you want less data
                tweetObj = json.loads(line)
                currentTime = datetime.datetime.strptime(tweetObj['postedTime'], timeFormat)
                id2=int(re.findall('^.*:([0-9]+)$',str(tweetObj['actor']['id']))[0])
                G.add_node(id2, parent=True, child=False)
                for ui in tweetObj['twitter_entities']['user_mentions']:
                    id1=ui['id']
                    G.add_node(id1, child=True)
                    G.add_edge(id1,id2, posted=currentTime, message=tweetObj['body'])
                    #print id1, id2
                try:
                    if (not tweetObj['body'].lower().startswith("rt")):
                        # Increment tweet count
                        globalTweetCounter += 1
                        tweetList.append(tweetObj['body'].lower())
                        timeList.append(currentTime)
                        userList.append(tweetObj['actor']['id'])
                        #print globalTweetCounter, tweetObj['body'].lower()
                except:
                    pass
print globalTweetCounter

90502


In [175]:
print np.min(timeList)
print np.max(timeList)
split = datetime.datetime(2014, 5, 15) #train/test split

2014-03-15 04:26:31
2014-06-13 22:05:22


In [176]:
print G.number_of_edges(), G.number_of_nodes(), len(timeList)

173629 26669 90502


In [177]:
for node in G.nodes():
    nTrain = 0
    nTest = 0
    for nbr in G.neighbors(node):
        if G.edge[node][nbr]['posted'] < split:
            nTrain += 1
        else:
            nTest += 1
    if nTrain < 3 or nTest < 3:
        G.remove_node(node)

In [178]:
G_train = G.copy()

for u,v in G_train.edges():
    if G_train[u][v]['posted'] > split:
        G_train.remove_edge(u,v)


In [179]:
print("Full graph: %d edges %d nodes" % (G.number_of_edges(), G.number_of_nodes()))
print("Training graph: %d edges %d nodes" % (G_train.number_of_edges(), G_train.number_of_nodes()))

Full graph: 41215 edges 2367 nodes
Training graph: 29830 edges 2367 nodes


In [180]:
jacDict = {}
adamDict = {}
nbrDict = {}
attDict = {}
spDict = {}
for node in G_train.nodes():
    jacDict[node] = {}
    adamDict[node] = {}
    nbrDict[node] = {}
    attDict[node] = {}
    spDict[node] = {}
    
def get_sp(u, v):
    (u,v) = sorted((u,v))
    ed = None
    if G_train.has_edge(u,v):
        ed = G_train.edge[u][v]
        G_train.remove_edge(u,v)
    if v not in spDict[u]:
        try:
            spDict[u][v] = nx.shortest_path_length(G_train, u, v)
        except:
            spDict[u][v] = 1000000
    
    if ed:
        G_train.add_edge(u, v, ed)
    return spDict[u][v]
            
    
def get_jac(u, v):
    (u,v) = sorted((u,v))
    if v not in jacDict[u]:
        j = nx.jaccard_coefficient(G_train, [(u, v)])
        for x,y,p in j:
            jacDict[u][v] = p
    return jacDict[u][v]

def get_adam(u, v):
    (u,v) = sorted((u,v))
    if v not in adamDict[u]:
        j = nx.adamic_adar_index(G_train, [(u, v)])
        try:
            for x,y,p in j:
                adamDict[u][v] = p
        except:
            adamDict[u][v] = 0
    return adamDict[u][v]

def get_adam2(u,v):
    (u,v) = sorted((u,v))
    score = 0
    if v not in adamDict[u]:
        for nbr in nx.common_neighbors(G_train, u, v):
            deg = nx.degree(G_train, nbr)
            if deg >= 2:
                score += 1/math.log(deg)
        adamDict[u][v] = score
    return adamDict[u][v]

def get_att(u,v):
    (u,v) = sorted((u,v))
    if v not in attDict[u]:
        j = nx.preferential_attachment(G_train, [(u, v)])
        for x,y,p in j:
            attDict[u][v] = p
    return attDict[u][v]

def get_nbrs(u, v):
    (u,v) = sorted((u,v))
    if v not in nbrDict[u]:
        nbrs = 0
        for nbr in nx.common_neighbors(G_train, u, v): nbrs += 1
        nbrDict[u][v] = nbrs
    return nbrDict[u][v]
        
def all_pairs(graph):
    return chain(graph.edges(), nx.non_edges(graph))

In [181]:
for u,v in all_pairs(G_train):
    print get_sp(u,v)

In [182]:
progress = 0

for n1, n2 in all_pairs(G_train):
    progress += 1
    if progress % 1000000 == 0: print progress
    get_nbrs(n1, n2)
    get_jac(n1, n2)
    get_adam2(n1, n2)
    get_att(n1, n2)
    get_sp(n1, n2)
        

1000000
2000000


In [188]:
u = []
v = []
has_links = []
jac_co = []
adam = []
att = []
nbrs = []
spl = []
count = 0
for n1, n2 in all_pairs(G_train):
    count += 1
    u.append(n1)
    v.append(n2)
    has_links.append(G_train.has_edge(n1,n2))
    jac_co.append(get_jac(n1,n2))
    adam.append(get_adam2(n1, n2))
    att.append(get_att(n1, n2))
    nbrs.append(get_nbrs(n1, n2))
    spl.append(get_sp(n1, n2))
                
df_train = pd.DataFrame()
df_train['u'] = u
df_train['v'] = v
df_train['link'] = has_links
df_train['jac'] = jac_co
df_train['adam'] = adam
df_train['nbrs'] = nbrs
df_train['att'] = att
df_train['spl'] = spl
print("%d pairs in training set, %d edges" % (count, np.count_nonzero(has_links)))

2800626 pairs in training set, 29830 edges


In [189]:
u = []
v = []
has_links = []
jac_co = []
adam = []
att = []
nbrs = []
spl = []
count = 0

for n1, n2 in nx.non_edges(G_train):
    u.append(n1)
    v.append(n2)
    has_links.append(G.has_edge(n1,n2))
    jac_co.append(get_jac(n1,n2))
    adam.append(get_adam2(n1, n2))
    att.append(get_att(n1, n2))
    nbrs.append(get_nbrs(n1, n2))
    spl.append(get_sp(n1, n2))
    count += 1

df_test = pd.DataFrame()
df_test['u'] = u
df_test['v'] = v
df_test['link'] = has_links
df_test['jac'] = jac_co
df_test['adam'] = adam
df_test['att'] = att
df_test['nbrs'] = nbrs
df_test['spl'] = spl
print("%d pairs in test set, %d true edges" % (count, np.count_nonzero(has_links)))

2770796 pairs in test set, 11205 true edges


In [ ]:
rf = RandomForestClassifier(n_estimators=1000, max_depth=None,
    min_samples_split=2, random_state=0, )
#rf = SVC(C=.00001, gamma=100, probability=True)
fields = ['att']
x_train = df_train.loc[:, fields]
y_train = np.reshape(df_train.link, (-1, 1))

x_test = df_test.loc[:, fields]
classifier = rf.fit(x_train, y_train)
pred = classifier.predict_proba(x_test)

/Users/tomfw/Library/Python/2.7/lib/python/site-packages/ipykernel_launcher.py:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if __name__ == '__main__':


In [ ]:
roc_auc_score(np.reshape(df_test.link, (-1, 1)), pred[:,1])

In [202]:
counts = 0
corrects = 0
for i in range(0, len(pred[:,1])):
    if pred[i,1] > .5:
        counts += 1
        if G.has_edge(df_test.u[i], df_test.v[i]):
            corrects += 1

print("%d predicted... %d correct" %(counts, corrects))

1970 predicted... 565 correct


In [195]:
print pred[:, 1]

[ 0.01594619  0.01594619  0.01594619 ...,  0.00103429  0.00103429
  0.00103429]


In [ ]:
#all information about netwrok
## of tweet / time histogram
#randomly delete and predict missing edges
#epstein KATZ
#distribution of degree (log of degree) should look linear
#t-SNE
#approximate katz